In [4]:
# Import necessary packages
import pandas as pd
import numpy as np

### 1. IMPORT FORM DATA

In [5]:
# Import data
df = pd.read_csv("greyhound_database.csv", encoding = "ISO-8859-1") 

#### View the data

In [6]:
df.head()

,Name,Date,Track,Dis,Trp,Split,Bends,Fin,By,Win/Sec,Remarks,WnTm,Gng,Wght,SP,Grade,CalTm
0,Snowdon Legend,23/04/2016,Pelaw,435m,[2],5.55,1111,1st,1Â½,Ballyroyal Tide,"EP,Crd3,Ld3/4",26.33,10,31.0,6/4F,A3,26.43
1,Snowdon Legend,15/04/2016,Pelaw,435m,[3],5.66,1111,1st,4Â¾,Glengiblin Cecil,"QAw,ALd",26.75,-40,30.9,5/2C,A3,26.35
2,Snowdon Legend,25/03/2016,Pelaw,435m,[3],5.62,5455,6th,7Â½,Any Lace,"SAw,Crd2",26.07,10,31.5,3/1,A2,26.77
3,Snowdon Legend,18/03/2016,Pelaw,435m,[3],5.64,NaN,NaN,NaN,Solo,Mid-Rls,26.91,-40,31.7,NaN,T1,26.51
4,Snowdon Legend,10/01/2016,Pelaw,435m,[1],5.77,4556,6th,10Â¾,Ardera Buzz,LckdEP,26.30,N,31.6,7/2,A2,27.16


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83172 entries, 0 to 83171
Data columns (total 17 columns):
Name       83172 non-null object
Date       83172 non-null object
Track      83172 non-null object
Dis        83172 non-null object
Trp        83172 non-null object
Split      78924 non-null float64
Bends      77711 non-null object
Fin        79387 non-null object
By         78853 non-null object
Win/Sec    83138 non-null object
Remarks    81886 non-null object
WnTm       83172 non-null float64
Gng        83172 non-null object
Wght       52509 non-null float64
SP         70214 non-null object
Grade      83155 non-null object
CalTm      83172 non-null float64
dtypes: float64(4), object(13)
memory usage: 10.8+ MB


In [8]:
print("COLUMN              DATATYPE       EXAMPLE")
print("----------------------------------------------------------------------")
for col in df.columns:
    print(col," "*(18-len(col)),df[col].dtype," "*(8-len(df[col].dtype)),df.iloc[3940][col])

COLUMN              DATATYPE       EXAMPLE
----------------------------------------------------------------------
Name                object          Tenacious Ben
Date                object          05/03/2016
Track               object          Cryfd
Dis                 object          540m
Trp                 object          [2]
Split               float64          13.16
Bends               object          4333
Fin                 object          3rd
By                  object          5Â¾
Win/Sec             object          Westmead Ellie
Remarks             object          Chl2-1/2
WnTm                float64          34.26
Gng                 object          20
Wght                float64          32.6
SP                  object          9/4F
Grade               object          S4
CalTm               float64          34.91


### 2. DATA CLEANING 
convert string values to numerics where needed, alter certain values, create new columns

### Date column to datetime object
Date stored as standard pandas object, convert to datetime

In [9]:
df['Date'] = pd.to_datetime(df['Date'].str.strip(), format='%d/%m/%Y')

### Delete Non-Runner Records
Some rows exist for occasions when the greyhound did not run in the race for whatever reason. These will be deleted.

In [10]:
print("Total No. of records",len(df))
df = df[(df['Fin'] != "NR") & (df['Fin'] != None)]# delete useless rows
print("Total No. of records",len(df))

Total No. of records 83172
Total No. of records 82652


#### SOME COLUMNS WERE STORED AS NUMERICAL VALUES AND DID NOT NEED ANY FURTHER WORK
#### Split
The time to the first bend, bend_1
#### WnTm
The finish time of race winner
#### CalTm
Finish time of dog in question
#### Wght
weight

### Trap
Almost all UK greyhound races have 6 runners, with traps from 1 to 6. these will be stored as numerics.

In [11]:
vals=[]
for val in df["Trp"].unique():
    vals.append(val) 
print("Datatype:",df["Trp"].dtype)
print("Values:",vals)

Datatype: object
Values: ['[2]', '[3]', '[1]', '[4]', '[6]', '[5]', '[]', '[7]', '[8]', '[0]']


In [12]:
def numerize_trp(field_value):
    try:
        return float("".join(i for i in field_value if i.isdigit()))
    except:
        return None

In [13]:
df["Trp"] = df["Trp"].apply(numerize_trp) # convert column

In [14]:
for val in df["Trp"].unique():
    print(val)   

2.0
3.0
1.0
4.0
6.0
5.0
nan
7.0
8.0
0.0


##### 0 trap?
Solution to 0 trap = replace with 3.5 (average of 1 to 6)

In [15]:
df.loc[df['Trp'] == 0] 

,Name,Date,Track,Dis,Trp,Split,Bends,Fin,By,Win/Sec,Remarks,WnTm,Gng,Wght,SP,Grade,CalTm
47393,Townsend Rebel,2015-03-09,Derry,274m,0.0,NaN,NaN,1st,NaN,NaN,NaN,17.34,N,NaN,NaN,GR,17.34
50148,Mullrook Wegirl,2015-03-16,Derry,457m,0.0,NaN,NaN,1st,NaN,NaN,NaN,28.70,N,NaN,NaN,GR,28.70
54840,Pennys Mountain,2015-04-24,Clnml,274m,0.0,NaN,NaN,NaN,13,Solo,NaN,16.56,N,NaN,NaN,T1,16.56
75916,So Pearl,2015-10-17,DrumPk,306m,0.0,NaN,NaN,2nd,NaN,Santro Flash,NaN,18.49,-15,NaN,NaN,GR,0.00


In [16]:
for ind, row in df.iterrows(): # replace 0 trap with 3.5, the average of 1 to 6
    val = row["Trp"]
    if val == 0:
        df.at[ind,"Trp"] = 3.5 

##### 7 and 8 trap?
only the track "Towc" has races with traps 7 and 8. Only 94 examples. This should not be a problem.

In [17]:
df.loc[(df['Trp'] == 7) | (df['Trp'] == 8)]

,Name,Date,Track,Dis,Trp,Split,Bends,Fin,By,Win/Sec,Remarks,WnTm,Gng,Wght,SP,Grade,CalTm
26953,Farran Williams,2017-01-28,Towc,260m,7.0,NaN,5-7-,7th,7Â¾,Grafton Willow,"SAw,NvrShwd,",16.33,-30,25.4,5/1,D3,16.65
26954,Farran Williams,2017-01-14,Towc,260m,8.0,NaN,1-7-,8th,7,Optimus Prime,"QAw,EvCh,",16.18,-30,25.2,5/1,D3,16.45
26955,Farran Williams,2017-01-07,Towc,260m,8.0,NaN,7-6-,6th,6Â¼,Head Iton Sandy,"SAw,W2,",16.06,-20,25.4,6/1,D3,16.35
26956,Farran Williams,2016-12-31,Towc,260m,8.0,NaN,2-5-,5th,4Â¼,Force Ten,"EP,W,ClrRn,",15.76,-15,25.2,9/2,D3,15.95
26958,Farran Williams,2016-12-17,Towc,260m,8.0,NaN,1-1-,1st,Â¾,Jet Stream Flyer,"QAw,ALd,",15.93,-15,25.4,5/1,D3,15.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82651,Trapstyle Yam,2017-04-15,Towc,260m,7.0,NaN,5-3-,5th,7Â¼,Hack Up Boy,VW2,15.64,N,29.7,8/1,D3,16.23
82652,Trapstyle Yam,2017-04-08,Towc,260m,7.0,NaN,NaN,NaN,NaN,No Race,NaN,0.00,N,30.3,NaN,D3,0.00
82656,Trapstyle Yam,2016-12-24,Towc,260m,8.0,NaN,1-2-,3rd,1Â¾,Near The Bowery,"QAw,EvCh,",16.06,-20,30.6,6/1,D3,15.99
82660,Trapstyle Yam,2016-11-12,Towc,260m,7.0,NaN,5-5-,3rd,4,Chantilly Lass,"W2,",15.90,-20,30.7,10/1,D3,16.01


In [18]:
vals=[]
for val in df["Trp"].unique():
    vals.append(val) 
print("Datatype:",df["Trp"].dtype)
print("Values:",vals)

Datatype: float64
Values: [2.0, 3.0, 1.0, 4.0, 6.0, 5.0, nan, 7.0, 8.0, 3.5]


### Distance: 'Dis'
removed the string characters such as 'm' and convert column to numeric.

In [19]:
vals=[]
for val in df["Dis"].unique():
    vals.append(val)
print("Datatype:",df["Dis"].dtype)
print("Values:",vals)

Datatype: object
Values: ['435m', '380m', '265m', '480m', '320m', '540m', '380mH', '285m', '450m', '515m', '415m', '592m', '545m', '375m', '714m', '500m', '695m', '274m', '400m', '457m', '297m', '240m', '302m', '329m', '366m', '480mH', '273m', '874m', '225m', '238m', '60mH', '642m', '525m', '462m', '261m', '460m', '277m', '475m', '660m', '555m', '640m', '840m', '718m', '630m', '445m', '503m', '575m', '330m', '280m', '500mH', '515mH', '540mH', '306m', '350m', '335m', '260m', '550m', '300m', '470m', '268m', '263m', '485m', '250m', '305m', '470mH', '275m', '686m', '685m', '290m', '245m', '420m', '620m', '258m', '264m', '428m', '526m', '400mH', '655m', '210m', '750m', '680m', '273mH', '220m', '692m', '700m', '509m', '549m', '768m', '285mH', 'mH', '0m', '659m', '416m', '590m', '668m', '521m', '60m', '265mH', '895m', '736m', '415mH', '663m', '483m', '740m', '764m']


In [20]:
def numerize_dis(field_value):
    try:
        return float("".join(i for i in field_value if i.isdigit()))
    except:
        #print(field_value)
        return None

In [21]:
df["Dis"] = df["Dis"].apply(numerize_dis) # convert column

In [22]:
vals=[]
for val in df["Dis"].unique():
    vals.append(val) 
print("Datatype:",df["Dis"].dtype)
print("Values:",vals)

Datatype: float64
Values: [435.0, 380.0, 265.0, 480.0, 320.0, 540.0, 285.0, 450.0, 515.0, 415.0, 592.0, 545.0, 375.0, 714.0, 500.0, 695.0, 274.0, 400.0, 457.0, 297.0, 240.0, 302.0, 329.0, 366.0, 273.0, 874.0, 225.0, 238.0, 60.0, 642.0, 525.0, 462.0, 261.0, 460.0, 277.0, 475.0, 660.0, 555.0, 640.0, 840.0, 718.0, 630.0, 445.0, 503.0, 575.0, 330.0, 280.0, 306.0, 350.0, 335.0, 260.0, 550.0, 300.0, 470.0, 268.0, 263.0, 485.0, 250.0, 305.0, 275.0, 686.0, 685.0, 290.0, 245.0, 420.0, 620.0, 258.0, 264.0, 428.0, 526.0, 655.0, 210.0, 750.0, 680.0, 220.0, 692.0, 700.0, 509.0, 549.0, 768.0, nan, 0.0, 659.0, 416.0, 590.0, 668.0, 521.0, 895.0, 736.0, 663.0, 483.0, 740.0, 764.0]


### Finish Position: 'Fin'

In [23]:
vals=[]
for val in df["Fin"].unique():
    vals.append(val) 
print("Datatype:",df["Fin"].dtype)
print("Values:",vals)

Datatype: object
Values: ['1st', '6th', nan, '2nd', '4th', '3rd', '5th', '7th', '8th', '=2nd', '=3rd']


In [24]:
def numerize_fin(field_value):
    try:
        return float("".join(i for i in field_value if i.isdigit()))
    except:
        #print(field_value)
        return None

In [25]:
df["Fin"] = df["Fin"].apply(numerize_fin) # convert column

In [26]:
vals=[]
for val in df["Fin"].unique():
    vals.append(val) 
print("Datatype:",df["Fin"].dtype)
print("Values:",vals)

Datatype: float64
Values: [1.0, 6.0, nan, 2.0, 4.0, 3.0, 5.0, 7.0, 8.0]


### Distance race won or lost by: 'By'

In [27]:
vals=[]
for val in df["By"].unique():
    vals.append(val) 
print("Datatype:",df["By"].dtype)
print("Values:",vals)

Datatype: object
Values: ['1Â½', '4Â¾', '7Â½', nan, '10Â¾', '4', '3Â¾', '10', '5Â¾', 'Â½', '9Â¼', '3Â½', '5Â¼', '2Â¼', '8Â½', 'nk', '6', '8Â¼', '5', '13Â¾', '4Â½', '10Â½', '1Â¾', '12Â¼', '9', '1', '12Â½', '9Â¾', '7Â¼', '6Â½', '16Â¾', '12', '7Â¾', '2Â¾', '6Â¼', 'hd', '3', 'Â¾', '1Â¼', '2', 'shd', '2Â½', '7', '15Â¾', 'dist', '8Â¾', '8', '3Â¼', '11Â¾', '6Â¾', '5Â½', '12Â¾', '4Â¼', '11', '18', '13Â¼', '13', '16Â¼', '14Â¼', '19', '11Â½', '14', '9Â½', '11Â¼', '14Â¾', 'DNF', '13Â½', '22Â½', '15', '17Â¼', '15Â½', '10Â¼', '16Â½', '16', '14Â½', '18Â½', '18Â¾', '17Â¾', '19Â¼', '15Â¼', '19Â½', '25Â¾', '19Â¾', '20', '17', '18Â¼', '20Â¼', '23Â¾', '25Â½', '17Â½', 'dht', '22Â¾', '20Â¾', '22', '48', '21Â¾', '23Â½', '23Â¼', '24', '27', '20Â½', '34Â½', '29Â½', '26Â½', '21Â½', '22Â¼', '43', '21', '24Â½', '21Â¼', '32Â¼', '30Â¾', '35', '24Â¼', '43Â½', '23', '28Â½', '28Â¾', '33Â¼', '33Â½', '26', '29Â¾', '26Â¾', '30', '25']


In [28]:
def numerize_by(field_value):
    try:
        return float("".join(i for i in field_value if i.isdigit()))
    except:
        #print(field_value)
        return field_value

In [29]:
df["By"] = df["By"].apply(numerize_by) # convert column

In [30]:
vals=[]
for val in df["By"].unique():
    vals.append(val) 
print("Datatype:",df["By"].dtype)
print("Values:",vals)

Datatype: object
Values: [1.0, 4.0, 7.0, nan, 10.0, 3.0, 5.0, 'Â½', 9.0, 2.0, 8.0, 'nk', 6.0, 13.0, 12.0, 16.0, 'hd', 'Â¾', 'shd', 15.0, 'dist', 11.0, 18.0, 14.0, 19.0, 'DNF', 22.0, 17.0, 25.0, 20.0, 23.0, 'dht', 48.0, 21.0, 24.0, 27.0, 34.0, 29.0, 26.0, 43.0, 32.0, 30.0, 35.0, 28.0, 33.0]


In [31]:
import numbers
for val in vals:
    if isinstance(val, numbers.Number):
        pass
    else:
        print(val)

Â½
nk
hd
Â¾
shd
dist
DNF
dht


In [32]:
df.loc[df['By'] == "Â½"] # It appears to be a short distance so will convert to 0.5 for 1/2 metre

,Name,Date,Track,Dis,Trp,Split,Bends,Fin,By,Win/Sec,Remarks,WnTm,Gng,Wght,SP,Grade,CalTm
11,Snowdon Legend,2015-05-28,Cryfd,380.0,2.0,3.81,6332,1.0,Â½,Lucozade Danni,"MsdBrk,LdNrLn",23.90,N,31.3,4/1,A2,23.90
45,Snowdon Legend,2014-07-26,Nwbrdg,480.0,3.0,0.95,NaN,2.0,Â½,Deanos Legend,"Ld1/2,EvCh",29.68,N,NaN,4/7F,A5,29.71
131,Good Fifi,2015-04-07,Cryfd,380.0,6.0,3.68,2111,1.0,Â½,Fairest Belle,"EP,SnLd",23.75,N,24.4,EvsF,A6,23.75
305,Clash Tenor,2014-06-13,Cryfd,380.0,5.0,3.70,3222,1.0,Â½,Millbrook Dance,"Crd1,Bmp&LdNrLn",24.40,-20,NaN,NaN,T3,24.20
318,Wajas Speckle,2016-01-02,Cryfd,380.0,4.0,3.59,1111,1.0,Â½,Suirview Zippy,"QAw,ALd",23.98,20,31.7,7/4F,A6,24.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82973,Winning Jasper,2016-02-20,Cryfd,380.0,5.0,3.57,2111,1.0,Â½,Dromscarra Brae,"EP,Disp-Ld3,Bmp1/4",23.73,20,33.3,5/1,A6,23.93
82983,Winning Jasper,2015-10-09,Clnml,480.0,5.0,NaN,NaN,1.0,Â½,Slanestown Echo,NaN,30.09,N,33.2,NaN,T2,30.09
83005,Yahoo Esther,2017-11-09,Cryfd,380.0,1.0,3.49,1111,1.0,Â½,Actually Blackie,"EP,SnLd",23.76,10,26.8,3/1,A3,23.86
83078,Yahoo Esther,2016-04-02,Cryfd,380.0,1.0,3.52,1111,1.0,Â½,Tonetta Tiger,"QAw,ALd",23.43,30,25.3,3/1,A5,23.73


In [33]:
for ind, row in df.iterrows(): # All are short distances approximately 0.25 metres should work
    val = row["By"]
    if val == 'nk' or val == 'hd' or val == 'shd' or val == 'Â¾':
        df.at[ind,"By"] = 0.25

In [34]:
for ind, row in df.iterrows(): # 0.5 metres should work
    val = row["By"]
    if val == 'Â½':
        df.at[ind,"By"] = 0.5

In [35]:
for ind, row in df.iterrows(): # All are long distances DNF=did not finish, dist = a significant distances
    val = row["By"]
    if val == 'DNF' or val == '' or val == 'dist':
        df.at[ind,"By"] = 50.0 # Will replace with 50 metres as this is longest distance recorded in all other examples.

In [36]:
df.loc[df['By'] == "dht"] # dht, It represents dead heat, that's where 2 or more greyhounds finish joint first

,Name,Date,Track,Dis,Trp,Split,Bends,Fin,By,Win/Sec,Remarks,WnTm,Gng,Wght,SP,Grade,CalTm
5610,Suncroft Rosie,2016-07-12,Sland,450.0,3.0,5.09,1111,1.0,dht,Russmur Prince,LdRnUp,27.69,N,25.7,6/1,A3,27.69
6886,Blackrose Linda,2013-07-20,Hnlow,460.0,3.0,3.89,2211,1.0,dht,Ardera Maureen,"Ld3,HldOn",28.92,10,NaN,2/1,A10,29.02
8964,Ballycannon Ruby,2013-11-12,Cryfd,380.0,5.0,3.66,1222,1.0,dht,Millroad Maldini,"Bmp1,LdNrLn",24.12,N,NaN,9/2,A5,24.12
16751,Knockmoy Jack,2016-08-16,Cryfd,380.0,5.0,3.60,4442,1.0,dht,Geneva Danny,Ld3/4,23.76,10,32.6,5/1,A3,23.86
17498,Village Blackie,2014-10-04,Cryfd,380.0,5.0,3.62,3322,1.0,dht,High St Destiny,"EP,Disp-Bmp1,LdNrLn",23.77,10,NaN,3/1,A3,23.87
17601,High St Destiny,2014-10-04,Cryfd,380.0,6.0,3.61,4111,1.0,dht,Village Blackie,"EP,Ld1",23.77,10,NaN,4/1,A3,23.87
23231,Barrack Bound,2014-05-10,Cryfd,540.0,3.0,13.15,1111,1.0,dht,Rileys Baby,"EP,SnLd",34.93,20,NaN,9/4,S7,35.13
25888,Dromscarra Brae,2017-12-15,Cryfd,540.0,4.0,13.25,2443,1.0,dht,Really O Riley,"LdNrLn,Mid",34.25,30,26.1,7/2,S4,34.55
26981,Farran Williams,2016-06-17,Cryfd,380.0,6.0,3.54,2111,1.0,dht,Boots Of Pride,"EP,SnLd",23.57,30,25.4,8/1,A4,23.87
27466,Frosty Prince,2012-04-07,Sheff,500.0,3.0,4.46,3111,1.0,dht,Jonny Reb,"EP,Ld1",30.20,10,NaN,2/1F,A4,30.30


In [37]:
for ind, row in df.iterrows(): # 0 metres is used for dht (dead heat)
    val = row["By"]
    if val == 'dht':
        df.at[ind,"By"] = 0.0

In [38]:
df["By"] = pd.to_numeric(df["By"]) # Column was not auto converted so conversion required

In [39]:
vals=[]
for val in df["By"].unique():
    vals.append(val) 
print("Datatype:",df["By"].dtype)
print("Values:",vals)

Datatype: float64
Values: [1.0, 4.0, 7.0, nan, 10.0, 3.0, 5.0, 0.5, 9.0, 2.0, 8.0, 0.25, 6.0, 13.0, 12.0, 16.0, 15.0, 50.0, 11.0, 18.0, 14.0, 19.0, 22.0, 17.0, 25.0, 20.0, 23.0, 0.0, 48.0, 21.0, 24.0, 27.0, 34.0, 29.0, 26.0, 43.0, 32.0, 30.0, 35.0, 28.0, 33.0]


In [40]:
for ind, row in df.iterrows(): # 0 metres is used for dht (dead heat)
    finished = row["Fin"]
    by = row["By"]
    if finished > 1:
        df.at[ind,"By"] = by*-1

In [41]:
vals=[]
for val in df["By"].unique():
    vals.append(val) 
print("Datatype:",df["By"].dtype)
print("Values:",vals)

Datatype: float64
Values: [1.0, 4.0, -7.0, nan, -10.0, -4.0, -3.0, -5.0, 0.5, -9.0, -2.0, -8.0, 0.25, -6.0, -13.0, -12.0, 2.0, 7.0, 6.0, -0.5, -16.0, 12.0, -1.0, 3.0, 15.0, -50.0, -11.0, -0.25, 13.0, -18.0, 5.0, -14.0, -19.0, 8.0, -22.0, -15.0, -17.0, 9.0, 50.0, 10.0, 18.0, 25.0, -20.0, 11.0, -23.0, -25.0, 0.0, 20.0, 19.0, 14.0, -48.0, 17.0, 21.0, 16.0, 23.0, -21.0, -24.0, 27.0, -34.0, -29.0, -26.0, 43.0, 22.0, -32.0, 30.0, -35.0, -43.0, -28.0, -33.0, 33.0, -27.0, -30.0, 24.0]


#### Bends

In [40]:
vals=[]
for val in df["Bends"].unique():
    vals.append(val) 
print("Datatype:",df["Bends"].dtype)
print("Values:",vals)

Datatype: object
Values: ['1111', '5455', nan, '4556', '2222', '5444', '4566', '4444', '6444', '6332', '4466', '4433', '6355', '4223', '2111', '3666', '3444', '3344', '2333', '5555', '3555', '5-4-', '4325', '5666', '1112', '4225', '5344', '6533', '4555', '3334', '2223', '1122', '2146', '6555', '4312', '1333', '5554', '6333', '6644', '6544', '4544', '4222', '3533', '3544', '3333', '4322', '3554', '5443', '5111', '4543', '2211', '2544', '4321', '6566', '2655', '4333', '1555', '4434', '3534', '1655', '4332', '1222', '3543', '3111', '1334', '2555', '3311', '2443', '2666', '3443', '3211', '3656', '5655', '3422', '3665', '4666', '6656', '3222', '5333', '1434', '3322', '3326', '4554', '1211', '6543', '5466', '1566', '1313', '2335', '2444', '2322', '5432', '2433', '3655', '2665', '5645', '1454', '6666', '2224', '3345', '2566', '3221', '4655', '3335', '5545', '1232', '5544', '2332', '2345', '2455', '6665', '4533', '3456', '5334', '1444', '2334', '4455', '2112', '6554', '5664', '4233', '5433', '

In [41]:
def numerize_bends(field_value): # first convert each to numbers
    try:
        return float("".join(int(i) for i in field_value if i.isdigit()))
    except:
        return field_value

In [42]:
df["Bends"] = df["Bends"].apply(numerize_bends) # convert column

In [43]:
# Get bends seperately
def bend_1(field_value): 
    try:
        return int(str(field_value)[0])
    except:
        return None
    
def bend_2(field_value): 
    try:
        return int(str(field_value)[1])
    except:
        return None

def bend_3(field_value): 
    try:
        return int(str(field_value)[2])
    except:
        return None
    
def bend_4(field_value): 
    try:
        return int(str(field_value)[3])
    except:
        return None

In [44]:
df["Bend_1"] = df["Bends"].apply(bend_1) # new column
df["Bend_2"] = df["Bends"].apply(bend_2) # new column
df["Bend_3"] = df["Bends"].apply(bend_3) # new column
df["Bend_4"] = df["Bends"].apply(bend_4) # new column

In [45]:
df["Bend_1"] = pd.to_numeric(df["Bend_1"]) # Column was not auto converted so conversion required
df["Bend_2"] = pd.to_numeric(df["Bend_2"]) # Column was not auto converted so conversion required
df["Bend_3"] = pd.to_numeric(df["Bend_3"]) # Column was not auto converted so conversion required
df["Bend_4"] = pd.to_numeric(df["Bend_4"]) # Column was not auto converted so conversion required

In [46]:
df = df.drop(columns=["Bends"]) # Bends can now be dropped

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82652 entries, 0 to 83171
Data columns (total 20 columns):
Name       82652 non-null object
Date       82652 non-null datetime64[ns]
Track      82652 non-null object
Dis        82651 non-null float64
Trp        82638 non-null float64
Split      78924 non-null float64
Fin        78867 non-null float64
By         78853 non-null float64
Win/Sec    82618 non-null object
Remarks    81550 non-null object
WnTm       82652 non-null float64
Gng        82652 non-null object
Wght       52509 non-null float64
SP         70214 non-null object
Grade      82635 non-null object
CalTm      82652 non-null float64
Bend_1     77711 non-null float64
Bend_2     75980 non-null float64
Bend_3     77709 non-null float64
Bend_4     75964 non-null float64
dtypes: datetime64[ns](1), float64(12), object(7)
memory usage: 15.7+ MB


#### Racing Post SP (Starting Price (Odds))

In [48]:
vals=[]
for val in df["SP"].unique():
    vals.append(val) 
print("Datatype:",df["SP"].dtype)
print("Values:",vals)

Datatype: object
Values: ['6/4F', '5/2C', '3/1', nan, '7/2', '12/1', '6/1', '4/1', '20/1', '16/1', '8/1', '5/1', '9/2', '5/2', '14/1', '3/1C', '10/1', '2/1F', '9/4F', '7/4F', '4/7F', '2/1C', '2/1', '4/5F', '7/1', '5/2F', '2/1J', '5/2J', 'EvsF', '3/1J', '5/4F', '9/4', '9/4J', '3/1F', '7/2C', '11/4', '6/4', '5/4J', '6/4J', '7/4J', '25/1', '11/4J', 'Evs', '7/4', '10/11F', '11/8F', '11/4F', '40/1', '11/10F', '66/1', '50/1', '4/5', '4/6F', '8/13F', '5/4', '33/1', '4/1F', '11/4C', '9/4C', '7/2J', '4/1C', '11/2', '6/4C', '8/11F', '7/2F', '13/8F', '1/2F', '11/8J', '15/8F', '2/5F', '4/6', '2/5', '11/10', '5/1F', '1/1', '9/1', '4/9F', '11/1', '13/1', '15/2', '100/30', '13/2', '7/4C', '13/8', 'N/O', '100/1', '125/1', '1/2', '4/7', '17/2', 'N/P', 'EvsJ']


In [49]:
def numerize_sp(field_value): # Create Decimal Odds from the fractional odds stored as strings e.g. 2/1 or Evs
    try:
        return round(float(field_value),2)
    except:
        try:
            split_data = field_value.split("/") # Fractionals such as 2/1, 4/1, 5/2
            numer = float("".join(i for i in split_data[0] if i.isdigit()))
            denom = float("".join(i for i in split_data[1] if i.isdigit()))
            return round((numer/denom)+1,2)
        except:
            if 'Evs' in field_value: # For all types of evens odds (50/50)
                return 2.0
            return None

In [50]:
df["SP"] = df["SP"].apply(numerize_sp) # convert column

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82652 entries, 0 to 83171
Data columns (total 20 columns):
Name       82652 non-null object
Date       82652 non-null datetime64[ns]
Track      82652 non-null object
Dis        82651 non-null float64
Trp        82638 non-null float64
Split      78924 non-null float64
Fin        78867 non-null float64
By         78853 non-null float64
Win/Sec    82618 non-null object
Remarks    81550 non-null object
WnTm       82652 non-null float64
Gng        82652 non-null object
Wght       52509 non-null float64
SP         70210 non-null float64
Grade      82635 non-null object
CalTm      82652 non-null float64
Bend_1     77711 non-null float64
Bend_2     75980 non-null float64
Bend_3     77709 non-null float64
Bend_4     75964 non-null float64
dtypes: datetime64[ns](1), float64(13), object(6)
memory usage: 15.7+ MB


#### Race Grade

In [52]:
vals=[]
for val in df["Grade"].unique():
    vals.append(val) 
print("Datatype:",df["Grade"].dtype)
print("Values:",vals)

Datatype: object
Values: ['A3', 'A2', 'T1', 'T3', 'A1', 'OR', 'A4', 'A5', 'N1', 'T2', 'A6', 'A7', 'S6', 'A8', 'H3', 'S7', 'S5', 'S4', 'IT', 'GR', 'S3', 'S2', 'T4', 'S1', 'IV', 'S8', 'P3', 'P1', 'N2', 'P2', 'H2', 'H1', 'N3', 'E3', 'M1', 'E2', 'H4', 'D3', 'D2', 'A10', 'HP', 'E1', 'A9', 'A11', 'D4', 'D1', 'B6', 'B5', 'B4', 'B1', 'B2', 'B3', 'KS', 'S9', nan, 'HS1', 'M3', 'P5', 'H0', 'HD1', 'P7', 'P4', 'B7', 'B8', 'S10', 'T5', 'D5']


In [53]:
grades = ["A1","A2","A3","A4","A5","A6","A7","A8","A9","A10","A11"]
def grade_A_numeric(field_value):# Used to give numeric value(in new column) for A grade races 1 to 11
    try:
        if "A" in field_value:
            num = float("".join(i for i in field_value if i.isdigit()))
            return num
        else:
            return None
    except:
        None

In [54]:
df["Grade_A_Numeric"] = df["Grade"].apply(grade_A_numeric)

In [55]:
vals=[]
for val in df["Grade_A_Numeric"].unique():
    vals.append(val) 
print("Datatype:",df["Grade_A_Numeric"].dtype)
print("Values:",vals)

Datatype: float64
Values: [3.0, 2.0, nan, 1.0, 4.0, 5.0, 6.0, 7.0, 8.0, 10.0, 9.0, 11.0]


#### Going (Ground/Weather Conditions): Gng

In [56]:
vals=[]
for val in df["Gng"].unique():
    vals.append(val) 
print("Datatype:",df["Gng"].dtype)
print("Values:",vals)

Datatype: object
Values: ['10', '-40', 'N', '-20', '20', '-10', '30', '-30', '50', '-25', '-50', '15', '40', '-35', '-15', '-80', '-45', '-60', '-5', '-70', '60', '-160', '-100', '-90', '5', '-140', '70', '-150', '-120', '80', '-130', '25', '90', '-110', '-180', '-190', '-220', '35', '-75', '-55', '-170', '-105', '100']


In [57]:
def numerize_going(field_value): # first convert each to numbers
    try:
        return round(float(field_value),2)
    except:
        try:
            return float("".join(int(i) for i in field_value if i.isdigit()))
        except:
            if field_value == "N":
                return 0.0

In [58]:
df["Gng"] = df["Gng"].apply(numerize_going)

In [59]:
vals=[]
for val in df["Gng"].unique():
    vals.append(val) 
print("Datatype:",df["Gng"].dtype)
print("Values:",vals)

Datatype: float64
Values: [10.0, -40.0, 0.0, -20.0, 20.0, -10.0, 30.0, -30.0, 50.0, -25.0, -50.0, 15.0, 40.0, -35.0, -15.0, -80.0, -45.0, -60.0, -5.0, -70.0, 60.0, -160.0, -100.0, -90.0, 5.0, -140.0, 70.0, -150.0, -120.0, 80.0, -130.0, 25.0, 90.0, -110.0, -180.0, -190.0, -220.0, 35.0, -75.0, -55.0, -170.0, -105.0, 100.0]


#### New column: Num_Opps
The expected no. of competitors in a race (some will have nr unfortunately)

In [60]:
def num_opps(field_value): # This will give the expected number of opponents, generally 6 except for trials
    try:
        if field_value[0] == "T":
            num = float("".join(i for i in field_value if i.isdigit()))
            return num
        else:
            return 6
    except:
        return 6

In [61]:
df["Num_Opps"] = df["Grade"].apply(num_opps)

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82652 entries, 0 to 83171
Data columns (total 22 columns):
Name               82652 non-null object
Date               82652 non-null datetime64[ns]
Track              82652 non-null object
Dis                82651 non-null float64
Trp                82638 non-null float64
Split              78924 non-null float64
Fin                78867 non-null float64
By                 78853 non-null float64
Win/Sec            82618 non-null object
Remarks            81550 non-null object
WnTm               82652 non-null float64
Gng                82652 non-null float64
Wght               52509 non-null float64
SP                 70210 non-null float64
Grade              82635 non-null object
CalTm              82652 non-null float64
Bend_1             77711 non-null float64
Bend_2             75980 non-null float64
Bend_3             77709 non-null float64
Bend_4             75964 non-null float64
Grade_A_Numeric    53651 non-null float64
Num_Opp

#### Remarks
These are the comments section, which describe a greyhounds race

In [63]:
vals=[]
for val in df["Remarks"].unique():
    vals.append(val) 
print("Datatype:",df["Remarks"].dtype)
print("Values:",vals)

Datatype: object
Values: ['EP,Crd3,Ld3/4', 'QAw,ALd', 'SAw,Crd2', 'Mid-Rls', 'LckdEP', 'Rls-Mid,ClrRn', 'Mid,ClrRn', 'Bmp1,Crd4', 'Crd1&2&3', 'Bmp1/4&3&4', 'Crd1/4&2', 'MsdBrk,LdNrLn', 'BmpRnUp&1,Crd2&3', 'Crd1Rls', 'CrdRnUp&1/4', 'RnOn', 'Crd1', 'EP,SnLd', 'Crd1&2', 'Crd1/4', 'EP,Crd1&1/4&1/2', 'EvCh,Rls', 'MsdBrk,Crd1&2', 'Bmp1/4', 'CrdRnUp', 'BdCrd1', 'Crd1/4&2&3', 'BmpRnUp,Mid-Rls', 'MsdBrk,Bmp1/4,Crd3', 'EP,2ndFr1,Chl1/4', 'EP,Ld1/4-Bmp3', 'EP,Bmp1,Chl&Bmp3', 'BdCrd&W1/4', 'MsdBrk,Bmp1/4&3', 'MsdBrk,Crd3', 'BmpStt,Chl&Crd1/4,Crd2', 'BmpStt,Crd2,HitRls3', 'EP,Ld1', 'Ld1/4,Mid-Rls', 'EP,LdRnUp,Mid-Rls', 'Ld1/2,EvCh', 'Imp1,NvrShwd', 'QAw,RlsRnUp', 'QAw,EvCh', 'QAw,Fcd-Ck3', 'EvAw,A2nd', nan, 'Crd1&1/4,(ReQul)', 'MsdBrk,Crd1', 'Crd1,Ld2-Bmp3', 'EP,Disp-Crd1/4', 'EP,Ld1/4,Mid', 'BmpStt&2,Crd1/2', 'Crd1,RnOn', 'CrdStt,Bmp2,RnOn', 'Bmp1,Blk1/4', 'SAw,Crd3&4', 'MsdBrk,RnOn', 'BmpStt,LdNrLn', 'Bmp1/4,Crd4', 'Crd&W1/4', 'Bmp1,Crd1/4', 'EP,2ndFr1-1/4', 'EP,Crd&Ld1/4', '2ndFr1/4', 'Bmp1&1/4'

In [64]:
print("No. of different remarks/comments = ",len(set(vals)))

No. of different remarks/comments =  19810


#### Reduce No. Unique Remarks

In [65]:
new_vals = [] # Split Each comment type by comma(,) and get set of unique comments
for v in vals:
    try:
        y = v.replace('-',',').split(",")
        new_vals.extend(y)
    except:
        new_vals.append(v)

In [66]:
print(new_vals)

['EP', 'Crd3', 'Ld3/4', 'QAw', 'ALd', 'SAw', 'Crd2', 'Mid', 'Rls', 'LckdEP', 'Rls', 'Mid', 'ClrRn', 'Mid', 'ClrRn', 'Bmp1', 'Crd4', 'Crd1&2&3', 'Bmp1/4&3&4', 'Crd1/4&2', 'MsdBrk', 'LdNrLn', 'BmpRnUp&1', 'Crd2&3', 'Crd1Rls', 'CrdRnUp&1/4', 'RnOn', 'Crd1', 'EP', 'SnLd', 'Crd1&2', 'Crd1/4', 'EP', 'Crd1&1/4&1/2', 'EvCh', 'Rls', 'MsdBrk', 'Crd1&2', 'Bmp1/4', 'CrdRnUp', 'BdCrd1', 'Crd1/4&2&3', 'BmpRnUp', 'Mid', 'Rls', 'MsdBrk', 'Bmp1/4', 'Crd3', 'EP', '2ndFr1', 'Chl1/4', 'EP', 'Ld1/4', 'Bmp3', 'EP', 'Bmp1', 'Chl&Bmp3', 'BdCrd&W1/4', 'MsdBrk', 'Bmp1/4&3', 'MsdBrk', 'Crd3', 'BmpStt', 'Chl&Crd1/4', 'Crd2', 'BmpStt', 'Crd2', 'HitRls3', 'EP', 'Ld1', 'Ld1/4', 'Mid', 'Rls', 'EP', 'LdRnUp', 'Mid', 'Rls', 'Ld1/2', 'EvCh', 'Imp1', 'NvrShwd', 'QAw', 'RlsRnUp', 'QAw', 'EvCh', 'QAw', 'Fcd', 'Ck3', 'EvAw', 'A2nd', nan, 'Crd1&1/4', '(ReQul)', 'MsdBrk', 'Crd1', 'Crd1', 'Ld2', 'Bmp3', 'EP', 'Disp', 'Crd1/4', 'EP', 'Ld1/4', 'Mid', 'BmpStt&2', 'Crd1/2', 'Crd1', 'RnOn', 'CrdStt', 'Bmp2', 'RnOn', 'Bmp1', 'Blk1/4

In [67]:
print("No. of different remarks/comments = ",len(set(new_vals)))

No. of different remarks/comments =  2675


In [68]:
val_counts = []
for val in set(new_vals):
    c = 0
    for val_a in new_vals:
        if val_a == val:
            c+=1
    val_counts.append((val,c))

In [69]:
val_counts = sorted(val_counts, key=lambda x: x[1],reverse=True)

In [70]:
print(val_counts)

[('EP', 5045), ('MsdBrk', 2457), ('Mid', 1958), ('RnOn', 1644), ('SnLd', 1363), ('Rls', 1358), ('SAw', 1344), ('Bmp1', 1124), ('Crd1', 1083), ('W', 1058), ('Ld', 1057), ('Disp', 1054), ('QAw', 1044), ('Crd1/4', 945), ('Crd3', 914), ('Bmp1/4', 808), ('Crd2', 556), ('BmpRnUp', 538), ('1', 508), ('2ndFr1', 498), ('3', 470), ('Ld1', 457), ('2', 457), ('Bmp3', 454), ('RnIn', 454), ('1/2', 434), ('4', 422), ('2ndFr2', 386), ('Bmp2', 377), ('NrLn', 376), ('Crd4', 372), ('Ld2', 366), ('Crd1/2', 351), ('(ReQul)', 340), ('BmpStt', 339), ('Crd3/4', 316), ('Ld3', 300), ('(HT)', 285), ('LdRnUp', 279), ('2ndFr1/4', 270), ('Ld4', 257), ('3/4', 257), ('1/4', 256), ('Bmp4', 255), ('LdNrLn', 255), ('Ld1/2', 247), ('Ld1/4', 246), ('CrdRnUp', 242), ('W2', 231), ('LdRnIn', 226), ('Crd5', 223), ('W1/4', 221), ('Blk1', 216), ('Fcd', 215), ('CmAgn', 212), ('Bmp1/2', 212), ('BmpRnIn', 199), ('Ckd3', 172), ('Ld3/4', 163), ('ALd', 159), ('BdCrd1/4', 158), ('BdCrd1', 156), ('CrdRnIn', 155), ('Chl3', 143), ('MidRn

#### New column Wide
Wide Run denoted by W in remarks

In [71]:
rems = []
def wide_run(field_value): # This will give the expected number of opponents, generally 6 except for trials
    try:
        if "W" in field_value:
            rems.append(field_value)
            return 1
        else:
            return 0
    except:
        return 0

In [72]:
df["Wide"] = df["Remarks"].apply(wide_run)

In [73]:
print(set(rems))

{'QAw,Ld-W2,W4', 'EvCh,Mid-W', 'CrdStt&1&1/4&3,FnWll', 'EP,W,SnLd-4,(HT)', 'EP,SnLd-3,Mid-W', 'W,Ld1-4', 'Crd1&3/4,Mid-W', 'SAw,Bmp3,Crd1/2,FnWll', 'EP,Chl&W1/4,W3/4', 'Crd1/4,W2', 'EP,Bmp1,Ld3,VW4', 'Crd3B,W3/4', 'MsdBrk,Mid-W,RnOn', 'Mid-W,Outp', 'EP,SnLd,Crd1,Mid-W', 'CrdRnUp,Crd&W1/4', 'MsdBrk,W,Crd3/4', 'WStt,Crd2H', 'QAw,Ld-1,W', 'EP,SnLd-W1', 'SAw,W&Blk1/4BdBmp2', 'EP,Disp-W2,ChlFr3', 'W,VSAw,Crd3', 'WRnUp,Bmp1', 'W&Crd1/4,Crd3/4', 'Crd1,Ckd&W1/4', 'EP,Mid-W,Blk1/4,Ld-3/4', 'Bmp1/4,W3/4,LdNrLn', 'MsdBrk,Ld3/4,W', 'SnLd-W4,Bmp1/4,CmAgnNrLn', 'MsdBrk,WRnUp,RnOn', 'Crd1,FcdW2', 'EP,SnLd-1/2,W2,Crd3', 'SAw,Mid-W,VW2', 'SAw,W,Ld3', 'Mid-W,Bmp1,ChlFr3/4', 'Chl3-W4', 'Bmp3/4,W', 'SAw,Crd&W1/4,Crd4', 'W&BmpRnUp,W4', 'SAw,W,CrdRnIn,(HT)', 'Bmp1/2,Crd3/4,W', 'CrdRnUp&1,Mid-W', 'MsdBrk,EP,W1/4,BdCrd3', 'W&Crd1/4', 'SAw,Crd4,FnWll', 'SnLd,WnWll', 'EP,Ld1-2,W', 'Bmp1,Chl&W&Crd4', 'MsdBrk,Crd&Stmb1/4,W2', 'EP,SnLd-2,W1/4&3/4', 'Ld1/4-W4,CmAgn', 'Crd1/4,Ld4,Mid-W', 'EP,Ld1-W2,Bmp3', 'Ld-Bmp3,L

In [74]:
# Approximately 10% of runners go wide
print("Wide runner rate")
print(round(df["Wide"].sum()/len(df),2)*100,"%")

Wide runner rate
10.0 %


### EXPORT DATA

In [75]:
df.to_csv("greyhound_database_cleaned.csv", index=False, encoding='utf-8-sig')